In [2]:
import numpy as np
from copy import deepcopy
from icecream import ic

In [82]:
NUM_KNAPSACKS = 2
NUM_ITEMS = 10
NUM_DIMENSIONS = 2

In [83]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS) ## valori degli oggetti
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS)) ## pesi degli oggetti (n.b ogni peso ha più dimensioni e i constraint devono essere rispettati in ogni dimensione)
CONSTRAINTS = np.random.randint(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size= (NUM_KNAPSACKS, NUM_DIMENSIONS)) ## tante quante le dimensioni 

In [84]:
CONSTRAINTS
# ogni zaino è una riga e ogni colonna è il constraint sulla dimensione

array([[ 69,  28],
       [329,   7]], dtype=int32)

## TEST PROBLEMS

In [10]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = np.random.randint(0, 100, size=NUM_ITEMS) 
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS)) 
CONSTRAINTS = np.random.randint(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size= (NUM_KNAPSACKS, NUM_DIMENSIONS)) 
CONSTRAINTS

array([[524, 308],
       [142, 134],
       [241, 389]], dtype=int32)

In [68]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = np.random.randint(0, 1000, size=NUM_ITEMS) 
WEIGHTS = np.random.randint(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS)) 
CONSTRAINTS = np.random.randint(0, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size= (NUM_KNAPSACKS, NUM_DIMENSIONS)) 
CONSTRAINTS

array([[4244, 5801, 7617, 8020, 6910, 2233, 9125, 9096,  958, 4395],
       [ 688, 9492, 3977, 3724, 2903, 5283, 2239, 6657,  151, 8661],
       [1332, 5809, 4424, 2146, 2179, 8503, 3085, 3197, 6309, 8226],
       [3849, 2419, 5219, 9648, 4229, 8709, 7133, 7187, 1391, 7052],
       [2965, 2051, 8815, 2427, 3476, 5279, 3244,  897, 3454, 4820],
       [1510, 2520, 1577,   30, 2117, 5580, 3665, 3299, 4287, 1089],
       [5633, 3141, 6000, 8330, 4783,  867, 3536, 9266, 5213, 3382],
       [4288, 9418, 9951, 9056, 7996, 9856, 8500, 3208, 4863, 4504],
       [1353, 5539, 3632, 4294, 7512, 3748, 2262, 2664, 2966, 1260],
       [3824, 3079, 7131, 2628, 4435, 9777, 1302, 8938,  967, 9713]],
      dtype=int32)

In [12]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = np.random.randint(0, 1000, size=NUM_ITEMS) 
WEIGHTS = np.random.randint(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS)) 
CONSTRAINTS = np.random.randint(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size= (NUM_KNAPSACKS, NUM_DIMENSIONS)) 
CONSTRAINTS


array([[34779, 99162, 75811, ..., 66398, 21864, 46703],
       [35106, 65655, 98577, ..., 28531, 57470, 21450],
       [46257, 50160, 85191, ..., 40536, 69412, 49332],
       ...,
       [90196, 23818, 70840, ..., 39554, 71341, 98055],
       [46450, 87644, 71612, ..., 77939, 28051, 50490],
       [58128, 52928, 76776, ..., 15807, 10796, 50516]],
      shape=(100, 100), dtype=int32)

SOLUTION A

In [4]:
def enforce_unique_assignment(sol):
    for i in range(NUM_ITEMS):
        assigned = np.where(sol[:, i])[0]
        if len(assigned) > 1:
            keep = np.random.choice(assigned)
            sol[:, i] = False
            sol[keep, i] = True
    return sol

In [5]:


def tweak_non_valid_A(knapsacs:  list[set] ) -> list[set]:
    ## remove objects from all the knapsacks that are in overweight
    new_bags = deepcopy(knapsacs)
    #overweight knapsacks (bags)
    weights_per_bag = new_bags.astype(int).dot(WEIGHTS)
    overweight_mask = weights_per_bag > CONSTRAINTS
    overweight_bags = np.where(np.any(overweight_mask, axis=1))[0]

    for b in overweight_bags:
    
        items_in_bag = np.where(new_bags[b])[0]
        
        if len(items_in_bag) > 0:
            item_to_remove = np.random.choice(items_in_bag)
            new_bags[b][item_to_remove] = False 
    return new_bags

def tweak_valid_A(knapsack: list[set]) -> list[set]:
    ## it estract a knapsac and an element if it is on the knapsac it swap it with another element not in the knapsac
    new_bags = deepcopy(knapsack)

    b = np.random.randint(0, NUM_KNAPSACKS)
    item= np.random.randint(0, NUM_ITEMS)

    
    items_in_selected_bag = np.where(new_bags[b])[0]
    items_in_other_bags = np.where(np.any(new_bags[np.arange(NUM_KNAPSACKS) != b], axis=0))[0]
    
    if item in items_in_selected_bag:
        ## remove it from the selected knapsack
        new_bags[b][item] = False
        ## add a random item not in the selected knapsack 
        other_bags = [i for i in range(NUM_KNAPSACKS) if i != b]
        new_b = np.random.choice(other_bags)
        new_bags[new_b][item] = True
    else :
        ## add it to the selected knapsack
        new_bags[b][item] = True  
        if item in items_in_other_bags:
            ## remove from the knapsack that currently  has it
            knapsack_having_i = np.where(new_bags[:, item])[0][0]
            new_bags[knapsack_having_i][item] = False
            ## nothing to do if the item is not in any knapsack
    return new_bags

In [6]:
def cost(solution: list[set]) -> (float, float, bool):

    weights_per_bag = solution.astype(int).dot(WEIGHTS)  
    total_overweight = weights_per_bag - CONSTRAINTS
    total_value = np.sum(solution.astype(int).dot(VALUES))
    # validità: nessuna componente > 0
    valid = not np.any(total_overweight > 0)

    # penalità: somma delle eccedenze
    penalty = np.sum(np.maximum(0, total_overweight))
    # fitness: valore - penalità pesata
    alpha = 10
    fitness = total_value - alpha * penalty

    return fitness, penalty, valid

In [13]:
#random solution
current_solution = np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool)
for item in range(NUM_ITEMS):
    if np.random.random() < 0.5:  # probabilità di assegnare l'oggetto
        knapsack = np.random.randint(NUM_KNAPSACKS)
        current_solution[knapsack, item] = True
first_solution = deepcopy(current_solution)
first_solution


array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]], shape=(100, 5000))

In [ ]:


(current_fitness, current_cost, current_valid) = cost(current_solution)

MAX_STEPS = 1000


for steps in range(MAX_STEPS):

    ic(steps, current_cost, current_fitness, current_valid)
    if current_valid:
        ## valid solution
        new_solution = tweak_valid_A(current_solution)
        new_solution = enforce_unique_assignment(new_solution)
        (new_fitness, new_cost, new_valid) = cost(new_solution)
        if new_valid:
            if new_fitness>= current_fitness: 
                current_cost= new_cost
                current_fitness= new_fitness
                current_solution = new_solution
                current_valid= new_valid
                continue

    else: 
        ## invalid solution
        new_solution = tweak_non_valid_A(current_solution)
        new_solution = enforce_unique_assignment(new_solution)
        (new_fitness, new_cost, new_valid) = cost(new_solution)

        if not new_valid: 
            
        #still invalid solution 
            if new_cost < current_cost or (new_cost== current_cost and new_fitness>=current_fitness): 
                # new solution is closer to being valid
                current_cost = new_cost
                current_solution = new_solution
                current_fitness= new_fitness
                current_valid= new_valid
                continue
            else:
                continue
        else:
            ## new solution is valid so it's better 
            current_cost = new_cost
            current_solution = new_solution
            current_fitness= new_fitness
            current_valid= new_valid
            continue

    

print(current_cost, current_fitness, current_valid)



ic| steps: 0
    current_cost: np.int64(728995)
    current_fitness: np.int64(-6044962)
    current_valid: False
ic| steps: 1
    current_cost: np.int64(586220)
    current_fitness: np.int64(-4656091)
    current_valid: False
ic| steps: 2
    current_cost: np.int64(464133)
    current_fitness: np.int64(-3471847)
    current_valid: False
ic| steps: 3
    current_cost: np.int64(361862)
    current_fitness: np.int64(-2482849)
    current_valid: False
ic| steps: 4
    current_cost: np.int64(270775)
    current_fitness: np.int64(-1597469)
    current_valid: False
ic| steps: 5
    current_cost: np.int64(198289)
    current_fitness: np.int64(-901281)
    current_valid: False
ic| steps: 6
    current_cost: np.int64(144837)
    current_fitness: np.int64(-389451)
    current_valid: False
ic| steps: 7
    current_cost: np.int64(102347)
    current_fitness: np.int64(16405)
    current_valid: False
ic| steps: 8
    current_cost: np.int64(75003)
    current_fitness: np.int64(274569)
    current_vali

0 1042007 True


In [9]:
solution=current_solution
solution

array([[False, False, False, False,  True, False, False, False, False,
        False, False,  True, False, False, False, False,  True, False,
         True, False],
       [ True, False, False, False, False, False, False, False, False,
        False,  True, False,  True, False,  True, False, False,  True,
        False, False],
       [False,  True, False, False, False,  True, False,  True,  True,
        False, False, False, False,  True, False,  True, False, False,
        False,  True]])

In [10]:
# Check if the solution is valid
valid_per_bag = []
for b in range(NUM_KNAPSACKS):
    # seleziona gli item presenti nello zaino b
    items_in_b = np.where(solution[b])[0]            # indici degli item presenti
    total_weight = WEIGHTS[items_in_b].sum(axis=0)   # somma per dimensione
    valid_per_bag.append(np.all(total_weight <= CONSTRAINTS[b]))

overall_valid = all(valid_per_bag)
overall_valid

True

In [11]:
# Check that the same object does not appear in multiple knapsacks
np.all(solution.sum(axis=0) <= 1)

np.True_

In [12]:
def solution_evaluation(solution):
    total_value = 0
    percentage_weight_over_constraint = np.zeros((NUM_KNAPSACKS, NUM_DIMENSIONS))

    for b in range(NUM_KNAPSACKS): 
        knapsack_weight = np.zeros(NUM_DIMENSIONS)
        knapsack_value = 0
        for i in range(NUM_ITEMS):
            if solution[b][i]:
                knapsack_value += VALUES[i]
                knapsack_weight += WEIGHTS[i]
        total_value += knapsack_value
        percentage_weight_over_constraint[b] = knapsack_weight*100/CONSTRAINTS[b]


    return percentage_weight_over_constraint, total_value

solution_evaluation(solution)

(array([[97.56097561, 40.19448947],
        [26.30718954, 96.23655914],
        [90.09584665, 98.58356941]]),
 np.int32(766))

## comment:
After testing several approaches, I found that a more "explorative" tweak (tweak valid) function with fewer constraints performs better, as it helps avoid stagnation or early stalling. 